<img src="https://i1.wp.com/sublibanderas.com/wp-content/uploads/2021/04/kueski-pay-PNG.png?fit=455%2C130&ssl=1" align="center" width="400"/>

<div class="alert alert-block alert-warning">
    
## In this notebook we solve a Kaggle challenge for Kueski, focusing on feature engineering
    
</div>

<hr style="border:2px solid gray"> </hr>

# Import libraries

In [21]:
# Import libraries
import pandas as pd
import os
import datetime
import re
import time

In [22]:
# We take the time of start of the execution
start_time = time.time()

In [ ]:
#Seteamos para que no utilice notacion cientifica
pd.options.display.float_format = '{:.9f}'.format
#Seteo para que el máximo de columnas que muestra al levantar una base sean 500
pd.set_option('display.max_columns',500)
#Estos códigos hacen que la visualización de la consola abarque toda la pantalla (sin los recortes a los costados). Tambien hacen que al mostrar dataframes podamos ver todas las columnas que tiene.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Codigo para poder imprimir multiples outputs en una misma línea
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Build dataset and base features (using each register only)

## 1. Main table

In [23]:
# Load rating data (first 1M rows) --> BECAUSE THIS IS A CHALLENGE! IT'S ONLY A DEMO OF MY WORK 
df_rating = pd.read_csv('raw_data/rating.csv', nrows=1000000)
# Get date of rating from timestamp
df_rating['date'] = pd.to_datetime(df_rating['timestamp'].str[:10], format='%Y-%m-%d', errors='coerce')
# We create the target based on the business definition --> rating >=4 means 1 (or a 'high' rating)
df_rating['target'] = np.where(df_rating['rating']>=4, 1, 0) 
df_rating.head(3)

,userId,movieId,rating,timestamp,date,target
0,1,2,3.5,2005-04-02 23:53:47,2005-04-02,0
1,1,29,3.5,2005-04-02 23:31:16,2005-04-02,0
2,1,32,3.5,2005-04-02 23:33:39,2005-04-02,0


### 1.2 Features from Main table --> Not in ETL process

## Movie title & genre

In [24]:
# Load movie general data (title and genre)
df_movie = pd.read_csv('raw_data/movie.csv')
df_movie.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


### Movie general features:
* Movie Year
* Movie genre

In [25]:
# FEATURE: MOVIE YEAR
# We apply a regex (4 digits between parenthesis) to find the year of the movie
pattern = re.compile(r"\((\d{4})\)")
years_list = df_movie['title'].apply(lambda x: pattern.findall(x))
df_movie['movie_year']=[x[0] if len(x)==1 else np.nan for x in years_list]

In [26]:
# FEATURES DE GENEROS DE PELICULAS
# listamos los diferentes generos que hay en el dataframe
dif_genres=set(np.concatenate(df_movie['genres'].str.split('|')).ravel().tolist())
# Para cada genero posible crearemos una columna a modo de flag
for genre_i in dif_genres:
    df_movie['genre_'+genre_i.lower()] = np.where(df_movie['genres'].str.contains(genre_i), 1, 0)
    
# We are ready to drop the genre list column
df_movie.drop(columns='genres',inplace=True)
df_movie.head()

C:\Users\ruizf\AppData\Roaming\Python\Python37\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,movieId,title,movie_year,genre_film-noir,genre_documentary,genre_war,genre_horror,genre_thriller,genre_crime,genre_animation,...,genre_romance,genre_sci-fi,genre_adventure,genre_action,genre_fantasy,genre_mystery,genre_imax,genre_western,genre_drama,genre_children
0,1,Toy Story (1995),1995,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,0,0,1
1,2,Jumanji (1995),1995,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
2,3,Grumpier Old Men (1995),1995,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),1995,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,5,Father of the Bride Part II (1995),1995,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Load Tag data

In [27]:
# LOAD TAG DATA
# Load relevance of each tag for each movie
df_gen_scores = pd.read_csv('raw_data/genome_scores.csv')
# Load meaning of each tag
df_gen_tags = pd.read_csv('raw_data/genome_tags.csv')
# Merge the data 
df_gen = pd.merge(df_gen_scores, df_gen_tags, how='left', on='tagId')
df_gen.head(3)

,movieId,tagId,relevance,tag
0,1,1,0.02500,007
1,1,2,0.02500,007 (series)
2,1,3,0.05775,18th century


### Features with Tag Data

We'll reduce Tag data (1128 tags per movie is too much).So we will get:
* Top 5 most relevant tags of the movie
* Top 5 most relevant tags overall --> relevance of those for each movie
* Top 5 most discriminant tags overall --> relevance of those for each movie

In [28]:
# Problema: Para cada película tenemos 1128 tags. 
# Si esto lo cruzamos con nuestra tabla principal (ratings de userId a cada movieId) la base explota dimensionalmente
# (sumaríamos 1128 columnas, y generaría problemas predictivos --> features de casualidad predicen bien)
# por lo cual solamente tomaremos los top5 tags más importantes para cada una

# First we sort the data, based on relevance for each movie (descending), so we can get the first 5 of each movie
df_gen.sort_values(['movieId', 'relevance'], ascending=[True, False], inplace=True)
# Then we select, for each movie, the top 5 rows (tagIds)
df_top5_tags = df_gen.groupby('movieId').head(5)

In [29]:
# TAGS: MOST RELEVANT TAGS, & MOST DISCRIMINANT TAGS
# To define this, it is very important to understand de concept behind de "RELEVANCE" feature
# as the documentation is not very clear, I'll assume-->  relevance = how accurate the tag is for the movie 
# we define how many we'll take
top_n_tags_to_get = 5

# We group the data by tag and sumarize the relevancy, and also take the standard deviation that it has
df_tag_relevance = df_gen.groupby('tagId').agg({'relevance':['sum','mean','std']}).reset_index()
# Rename the columns just to be clearer
df_tag_relevance.columns = ['tagId','sum_relevance', 'mean_relevance','std_relevance']

# We add the tag description (meaning) 
df_tag_relevance = pd.merge(df_tag_relevance, df_gen_tags, how='left', on='tagId')

# We take the most relevant (sum of relevancies)
list_most_relevant_tags = df_tag_relevance.sort_values('sum_relevance', ascending=False).head(top_n_tags_to_get)['tagId'].values.tolist()
list_most_discrim_tags = df_tag_relevance.sort_values('std_relevance', ascending=False).head(top_n_tags_to_get)['tagId'].values.tolist()
# and their description
list_most_relevant_tags_desc = df_tag_relevance.sort_values('sum_relevance', ascending=False).head(top_n_tags_to_get)['tag'].values.tolist()
list_most_discrim_tags_desc = df_tag_relevance.sort_values('std_relevance', ascending=False).head(top_n_tags_to_get)['tag'].values.tolist()

In [30]:
# We build the features based on the former explanations
# 1) For each movie we get the TagId of it's most relevants tags

# we add order to the tags
df_top5_tags['order']=df_top5_tags.groupby('movieId')['relevance'].rank(method='first', ascending=False).astype(int)
# then we create a pivot table so there is only 1 row for each movie, and the columns have the top_n_tagId
df_tags_movies = pd.pivot_table(df_top5_tags, values='tag', index='movieId', columns='order', aggfunc=lambda x: ' '.join(x))
# df_tags_movies = pd.pivot_table(df_top5_tags, values='tagId', index='movieId', columns='order')
# change col names
df_tags_movies.columns = ['top_'+str(x)+'_tagId' for x in df_tags_movies.columns.tolist()]

# 2) We build columns for specific tags (top5 most relevant overall, and top5 most discriminant
# first we do it for the relevant tags
df_relevant_tags = pd.DataFrame(index=df_gen.movieId.unique())
for tag_i in list_most_relevant_tags_desc:
    df_relevant_tags['tag_'+str(tag_i)]=df_gen[df_gen['tag']==tag_i]['relevance'].values
# then we repeat the process for the most discriminant
df_discrim_tags = pd.DataFrame(index=df_gen.movieId.unique())
for tag_i in list_most_discrim_tags_desc:
    df_discrim_tags['tag_'+str(tag_i)]=df_gen[df_gen['tag']==tag_i]['relevance'].values 

# We gather all these features in one table
df_tag_features = pd.merge(df_tags_movies, df_relevant_tags, how='left', left_index=True, right_index=True)
df_tag_features = pd.merge(df_tag_features, df_discrim_tags, how='left', left_index=True, right_index=True)
df_tag_features.head(3)

C:\Users\ruizf\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,top_1_tagId,top_2_tagId,top_3_tagId,top_4_tagId,top_5_tagId,tag_original,tag_mentor,tag_great ending,tag_dialogue,tag_good soundtrack,tag_comedy,tag_action,tag_tense,tag_relationships,tag_horror
movieId,,,,,,,,,,,,,,,
1,toys,computer animation,pixar animation,kids and family,animation,0.87925,0.64925,0.6315,0.63750,0.73375,0.61800,0.67050,0.3335,0.11650,0.04725
2,adventure,jungle,children,kids,special effects,0.71900,0.61000,0.4295,0.37275,0.24625,0.54025,0.64600,0.1205,0.10125,0.06400
3,good sequel,sequel,sequels,comedy,original,0.77900,0.54575,0.4530,0.45600,0.31175,0.84675,0.16475,0.1045,0.31000,0.02400


## Load movie ids for external data

In [31]:
df_link = pd.read_csv('raw_data/link.csv')
df_link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


### Features external data (IMDB average rating and amount of them)

In [32]:
# Loads imdb rating data (from https://www.imdb.com/interfaces/)
df_imdb_ratings = pd.read_csv('raw_data/imdb_ratings.tsv', sep="\t")
# Deletes 'tt' and trailing zeros from the titleid (tconst)
df_imdb_ratings['tconst']=df_imdb_ratings['tconst'].str.replace('tt','').astype(int)
# We get the features only for the movies we have 
df_imdb_features=pd.merge(df_link[['movieId','imdbId']], df_imdb_ratings, how='left', left_on='movieId', right_on='tconst')
# We remove the imdbIDs because we already used them
df_imdb_features.drop(columns=['tconst','imdbId'],inplace=True)
# Change column names 
df_imdb_features.columns = ['movieId','imdb_avg_rating', 'imdb_qnt_votes']
df_imdb_features.head(3)

,movieId,imdb_avg_rating,imdb_qnt_votes
0,1,5.7,1847.0
1,2,6.0,240.0
2,3,6.5,1614.0


## Load tags applied to movies by users

In [33]:
df_user_tag = pd.read_csv('raw_data/tag.csv')
df_user_tag.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


## Output: 
* Main table with added features from: movie info, tags, imdb ratings
* Tags applied by users

In [34]:
# Add movie features to main dataframe
df_main = pd.merge(df_rating, df_movie, how='left', on='movieId' )
# Then we add tag features to the table
df_main = pd.merge(df_main, df_tag_features, how='left', left_on='movieId', right_index=True)
# Finally we add the external features (imdb ratings)
df_main = pd.merge(df_main,df_imdb_features, how='left', on='movieId')

### Export output

In [35]:
# Export main table as csv
df_main.to_csv('model_input/main_table.csv', sep=';', index=False)

In [36]:
# Export user_tags (unchanged) as csv
df_user_tag.to_csv('model_input/user_tags.csv', sep=';',index=False)

In [37]:
# Calculate total time of execution
end_time = time.time()
print(f"El script tarda {(end_time - start_time)/60} minutos en correr")

El script tarda 1.0394009868303935 minutos en correr
